# PROJECT - METHOD - YYMMDD - NPC MS Data Pipeline

#### This document provides a pipeline for the import of MS data (post peak-picking), and any associated sample metadata, followed by summaries and quality control reports of the data (both in sample and feature dimensions), implementation of batch correction and feature selection and output of a final dataset ready for sharing with collaborators and data modeling. See SOP # for further details of requirements, descriptions of expected outputs and options for optimising data quality.

# 1. Initial Setup

### Define file paths

In [ ]:
toolboxPath = '/local path to npyc-toolbox/phenomecentre/npyc-toolbox'
chemometricsPath = '/local path to pyChemometrics-toolbox'

QIdataPath = '/path to progenesis file/PROJECT dataset PIfile.csv'
rawDataPath = '/path to raw data/Data/'

limsFilePath = '/path to LIMS file/PROJECT dataset LIMSfile.csv'
manifestPath = '/path to subject information file/PROJECT SubjectINFOfile.csv'

saveDir = '/path to save directory/Projects/PROJECT/METHOD DATE/'

### Import code

In [ ]:
import os
import matplotlib.pyplot as plt
import scipy
import pandas
import numpy
import pickle
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
%matplotlib inline
import sys
sys.path.append(chemometricsPath)
sys.path.append(toolboxPath)
import nPYc
import copy
from nPYc.enumerations import VariableType, DatasetLevel, AssayRole, SampleType
from nPYc.utilities.normalisation import NullNormaliser, TotalAreaNormaliser, ProbabilisticQuotientNormaliser

In [ ]:
import datetime
from nPYc.__init__ import __version__ as version
print('Run with branch ' + version + ' on ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

### Create saveDir

In [ ]:
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
    os.makedirs(os.path.join(saveDir, 'data objects'))

# 2. Import Data & Sample Metadata

### Import acquired data and associated acqusition parameters

In [ ]:
msData = nPYc.MSDataset(QIdataPath)
msData.addSampleInfo(descriptionFormat='Filenames')
msData.addSampleInfo(descriptionFormat='Raw Data', filePath=rawDataPath)
msData.addSampleInfo(descriptionFormat='Batches')

### Match acquired samples to sample IDs (LIMS file) and subject information (if available)

In [ ]:
msData.addSampleInfo(descriptionFormat='NPC LIMS', filePath=limsFilePath)

In [ ]:
msData.addSampleInfo(descriptionFormat='NPC Subject Info', filePath=manifestPath)

### IF REQUIRED: save/load imported data

In [ ]:
# pickle.dump(msData, open(os.path.join(saveDir, 'data objects', msData.name + "_msDataImported.p"), "wb"))

In [ ]:
# msData = pickle.load( open('path to data objects/msDataImported.p', "rb"))

# 3. Sample & Feature Summary Reports

### Generate sample summary report

In [ ]:
nPYc.reports.generateReport(msData, 'sample summary')

### IF REQUIRED: remove samples marked for exclusion or of unknown type

In [ ]:
# To exclude any samples 'Marked for Exclusion' (e.g., samples with insufficient sample volume, injection failure):
# msData.excludeSamples(msData.sampleMetadata.iloc[msData.sampleMetadata['Skipped'].values==True]['Sample File Name'], on='Sample File Name', message='Marked for exclusion (_x)')

# To exclude any samples of 'Unknown' type:
# msData.excludeSamples(msData.sampleMetadata[pandas.isnull(msData.sampleMetadata['Sample Base Name'])]['Sample File Name'], on='Sample File Name', message='Unknown type')

# Then apply masks:
# msData.applyMasks()

### Generate feature summary report

In [ ]:
nPYc.reports.generateReport(msData, 'feature summary')

### IF REQUIRED: generate detailed correlation to dilution report

#### If correlation to dilution figures show sub-optimal results, the following report provides more detail in order that outlying samples/SRD batch subsets can be excluded from the correlation to dilution calculation.

In [ ]:
# nPYc.reports.generateReport(msData, 'correlation to dilution')

In [ ]:
# To generate an interactive plot for determining sample names of outlying samples:
# data = nPYc.plotting.plotTICinteractive(msData, plottype='Linearity Reference')
# iplot(data)

In [ ]:
# To exclude a complete subset of SRD samples, for example, B1LR1to46:
# msData.corrExclusions = ['B1LR1to46']

# To exclude specific samples, for example B2SRD83 and B2SRD84:
# msData.excludeSamples(msData.sampleMetadata.iloc[msData.sampleMetadata['Sample File Name'].values=='PROJECT_METHOD_ToF_B2SRD83']['Sample File Name'], on='Sample File Name', message='Outlying TIC SRD')
# msData.excludeSamples(msData.sampleMetadata.iloc[msData.sampleMetadata['Sample File Name'].values=='PROJECT_METHOD_ToF_B2SRD84']['Sample File Name'], on='Sample File Name', message='Outlying TIC SRD')

# Then apply masks:
# msData.applyMasks()

# 4. Batch & Run-Order Correction

#### The batch correction assessment report shows the TIC overall and the intensity and batch correction fit for a subset of randomly selected features, to check performance of batch correction and aid specification of additional batch start and end points or sample exclusions if required.

In [ ]:
nPYc.reports.generateReport(msData, 'batch correction assessment', batch_correction_window=11)

### IF REQUIRED: amend batches/exclude sample outliers

#### If there is a significant gap in sample aquisition, or a significant jump in intensity between adjacent samples quality of fit can be detrementally effected. In these cases, a new batch should be initialised at the first sample after the gap, or where intensity shifts. In addition, any study pool samples exhibiting an outlying TIC can also be excluded at this stage.

In [ ]:
# To generate an interactive plot for determining run order number of sample/s where a new batch should be started (number on second row):
# data = nPYc.plotting.plotTICinteractive(msData, plottype='Sample Type')
# iplot(data)

In [ ]:
# To define the start of new batch, for example at run order = 316:
# msData.amendBatches(316)

In [ ]:
# To exclude specific samples, for example sample with run order 1182:
# msData.excludeSamples(msData.sampleMetadata.iloc[msData.sampleMetadata['Run Order'].values==1181]['Sample File Name'], on='Sample File Name', message='Outlying TIC')

# Then apply masks:
# msData.applyMasks()

### Run batch correction

In [ ]:
msDatacorrected = nPYc.batchAndROCorrection.correctMSdataset(msData, parallelise=True)

### Generate batch correction summary report

In [ ]:
nPYc.reports.generateReport(msData, 'batch correction summary', msDataCorrected=msDatacorrected)

### IF NO BATCH CORRECTION REQUIRED: use original dataset

In [ ]:
# msDatacorrected = msData

# 5. Feature Filtering

#### By default features are filtered based on correlation to dilution, Residual Standard Deviation (RSD) and removal of artifactual features (highly correlated features with close RT and m/z). The feature selection report shows the number of features passing filtering with the default settings, and how this number would be altered if thresholds where changed. 

In [ ]:
nPYc.reports.generateReport(msDatacorrected, 'feature selection')

### IF REQUIRED: adjust feature filtering parameter thresholds

In [ ]:
# For example, to amend the RSD threshold: 
# msDatacorrected.Attributes['rsdThreshold'] = 20

# For example, to amend the correlation to dilution threshold:
# msDatacorrected.Attributes['corrThreshold'] = .8

### Apply feature selection

In [ ]:
msDatacorrected.updateMasks(filterFeatures=True, filterSamples=False)

### Save QC Reports for Review

In [ ]:
qcDir = os.path.join(saveDir, 'QC')
if not os.path.exists(qcDir):
    os.makedirs(qcDir)
nPYc.reports.generateReport(msDatacorrected, 'sample summary', destinationPath=qcDir)
nPYc.reports.generateReport(msDatacorrected, 'feature summary', withExclusions=True, destinationPath=qcDir)

# 6. Analytical Multivariate Quality Control

### Select the samples

In [ ]:
msDatacorrected.updateMasks(sampleTypes=[SampleType.StudySample, SampleType.StudyPool, SampleType.ExternalReference])

### Run analytical multivariate QC

#### There are several paramters in both the PCA model generation and multivariate report that can be optimised depending on your dataset, please see documentation for details. 

In [ ]:
# NOTE: default scaling=1 ('uv'); withExclusions=False (masks not applied)
PCAmodelAnalytical = nPYc.multivariate.exploratoryAnalysisPCA(msDatacorrected, withExclusions=True)

In [ ]:
nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelAnalytical, reportType='analytical', withExclusions=True)

### IF REQUIRED: generate interactive scores and loadings plots

In [ ]:
# Interactive scores plot, e.g., plotting the scores for the first two components coloured by run order

# data = nPYc.plotting.plotScoresInteractive(msDatacorrected, PCAmodelAnalytical, 'Run Order', components=[1, 2], withExclusions=True)
# iplot(data)

In [ ]:
# Interactive loadings plot, e.g., plotting the loadings for component 2

# data = nPYc.plotting.plotLoadingsInteractive(msDatacorrected, PCAmodelAnalytical, component=2, withExclusions=True)
# iplot(data)

### Save QC Reports for Review

In [ ]:
nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelAnalytical, reportType='analytical', withExclusions=True, destinationPath=qcDir)

# 8. Finalise & Export Dataset

### Select the samples (default is SampleType.StudySample and SampleType.StudyPool samples only)

In [ ]:
msDatacorrected.updateMasks()

### Generate PCA model with updated settings

In [ ]:
PCAmodelAnalytical = nPYc.multivariate.exploratoryAnalysisPCA(msDatacorrected, withExclusions=True)

In [ ]:
nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelAnalytical, withExclusions=True, reportType='analytical')

### IF REQUIRED: mark samples for exclusion based on multivariate QC results

In [ ]:
# For example, mark outlying samples for exclusion (e.g., from interactive scores plot)
# msData.excludeSamples(msData.sampleMetadata.iloc[msData.sampleMetadata['Sample File Name'].values=='PROJECT_METHOD_ToF_PxWx']['Sample File Name'], on='Sample File Name', message='Outlier in PCA scores')

In [ ]:
# Repeat PCA modelling
# PCAmodelAnalytical = nPYc.multivariate.exploratoryAnalysisPCA(msDatacorrected, withExclusions=True)
# nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelAnalytical, reportType='analytical', withExclusions=True)

### Check final dataset output if current masks applied

In [ ]:
nPYc.reports.generateReport(msDatacorrected, 'final report', withExclusions=True, pcaModel=PCAmodelAnalytical)

### Apply masks

In [ ]:
msDatacorrected.applyMasks()

### Export data

In [ ]:
# Export final dataset
msDatacorrected.exportDataset(destinationPath=saveDir)

In [ ]:
# Export final summary report
nPYc.reports.generateReport(msDatacorrected, 'final report', destinationPath=saveDir, pcaModel=PCAmodelAnalytical)

In [ ]:
# To export combined dataset (e.g., format for SIMCA)
msDatacorrected.exportDataset(destinationPath=saveDir, saveFormat='UnifiedCSV')

### IF REQUIRED: change normalisation

In [ ]:
# For total area normalisation
# msDatacorrected.Normalisation = TotalAreaNormaliser()

# For probabilistic quotient normalisation
msDatacorrected.Normalisation = ProbabilisticQuotientNormaliser()

### PCA of normalised dataset

In [ ]:
PCAmodelAnalytical_normalised = nPYc.multivariate.exploratoryAnalysisPCA(msDatacorrected)

In [ ]:
nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelAnalytical_normalised, reportType='analytical')

### Export normalised data

In [ ]:
normalisedDir = os.path.join(saveDir, 'Normalised data')
if not os.path.exists(normalisedDir):
    os.makedirs(normalisedDir)

In [ ]:
msDatacorrected.exportDataset(destinationPath=normalisedDir)

In [ ]:
nPYc.reports.generateReport(msDatacorrected, 'final report', pcaModel=PCAmodelAnalytical_normalised, destinationPath=normalisedDir)

In [ ]:
msDatacorrected.exportDataset(destinationPath=normalisedDir, saveFormat='UnifiedCSV')

# 9. Biological Multivariate Report

In [ ]:
# Keep study samples only, but all features
msDatacorrected.updateMasks(sampleTypes=[SampleType.StudySample], filterFeatures=False)

In [ ]:
PCAmodelBiological = nPYc.multivariate.exploratoryAnalysisPCA(msDatacorrected, withExclusions=True)

In [ ]:
nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelBiological, reportType='biological', withExclusions=True)

In [ ]:
# Save report (NOTE: check destinationPath directory correct for whether data normalised or not)
nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelBiological, reportType='biological', withExclusions=True, destinationPath=normalisedDir)

### IF REQUIRED: define subset of biological parameters, with defined type, for plotting

In [ ]:
# Define parameters to plot, keys as column names, values as data type
# biologicalMeasurements = {'Test': 'categorical', 'Age': 'continuous'}

# Repeat PCA
# PCAmodelBiological = nPYc.multivariate.exploratoryAnalysisPCA(msDatacorrected)
# nPYc.reports.multivariateQCreport(msDatacorrected, PCAmodelBiological, reportType='biological', withExclusions=True, biologicalMeasurements=biologicalMeasurements)